<a href="https://colab.research.google.com/github/afeld/python-public-policy/blob/main/lecture_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NYU Wagner - Python Coding for Public Policy**
# Class 5: Natural language processing (NLP)

# LECTURE

This lecture is more about showing what's possible with NLP than getting hung up on the specifics of how to do it.

But first, [a Twitter thread showing the power of data analysis](https://twitter.com/kate_ptrv/status/1332398737604431874).

In [40]:
%%html
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">I couldn’t just walk past this Tweet, so here is some fun <a href="https://twitter.com/hashtag/dataviz?src=hash&amp;ref_src=twsrc%5Etfw">#dataviz</a><br><br>Scented candles: An unexpected victim of the COVID-19 pandemic 1/n <a href="https://t.co/xEmCTQn9sA">https://t.co/xEmCTQn9sA</a> <a href="https://t.co/tVecEiX5Jc">pic.twitter.com/tVecEiX5Jc</a></p>&mdash; Kate Petrova (@kate_ptrv) <a href="https://twitter.com/kate_ptrv/status/1332398737604431874?ref_src=twsrc%5Etfw">November 27, 2020</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script> 

## Setup

We will be using [spaCy](https://spacy.io/) for NLP. First, we need to download the model:

In [1]:
!python -m spacy download en_core_web_md --quiet

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [2]:
import spacy

nlp = spacy.load("en_core_web_md")

## Concepts

- Model
- Training
- Document
- [Features](https://spacy.io/usage/spacy-101#features)
  - Tokens
  - Parts of speech
  - Lemmas
  - Entities
- Vectors

## Text structure

Example from [DataCamp](https://campus.datacamp.com/courses/advanced-nlp-with-spacy/finding-words-phrases-names-and-concepts?ex=4).

In [3]:
doc = nlp("In 1990, more than 60% of people in East Asia were in extreme poverty. Now less than 4% are.")

In [4]:
from spacy import displacy

displacy.render(doc)

This shows the [parts of speech](https://spacy.io/api/annotation#pos-tagging).

## Entities

In [5]:
displacy.render(doc, style='ent')

## Sentiment analysis

We are going to use a different library, [TextBlob](https://textblob.readthedocs.io/). Works similarly to spaCy, though.

First, need to [download the model](https://textblob.readthedocs.io/en/dev/install.html):

In [6]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /Users/afeld/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /Users/afeld/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/afeld/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/afeld/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /Users/afeld/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/afeld/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [7]:
from textblob import TextBlob

blob = TextBlob("I love this class!")
blob.sentiment

Sentiment(polarity=0.625, subjectivity=0.6)

In [8]:
blob2 = TextBlob("I hate pandas")
blob2.sentiment.polarity

-0.8

Let's get some real data to work with: [Public comments](https://beta.regulations.gov/document/DOT-OST-2018-0068-12959/comment) from the proposed [Traveling by Air with Service Animals](https://beta.regulations.gov/document/DOT-OST-2018-0068-12959) rule. We'll use a cleaned-up version.

In [10]:
import pandas as pd

pd.options.display.max_colwidth = None

df = pd.read_csv('https://nyu.box.com/shared/static/1zdvfmpy5452nqdlb5b1yho6m3bdihk9.csv')
df

id                  date  \
0    DOT-OST-2018-0068-14971  2020-02-24T05:00:00Z   
1    DOT-OST-2018-0068-14917  2020-02-24T05:00:00Z   
2    DOT-OST-2018-0068-14918  2020-02-24T05:00:00Z   
3    DOT-OST-2018-0068-14919  2020-02-24T05:00:00Z   
4    DOT-OST-2018-0068-14920  2020-02-24T05:00:00Z   
..                       ...                   ...   
493  DOT-OST-2018-0068-15072  2020-02-24T05:00:00Z   
494  DOT-OST-2018-0068-15074  2020-02-24T05:00:00Z   
495  DOT-OST-2018-0068-15075  2020-02-24T05:00:00Z   
496  DOT-OST-2018-0068-15078  2020-02-24T05:00:00Z   
497  DOT-OST-2018-0068-15080  2020-02-24T05:00:00Z   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          content  
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

How would you determine how many are for/against?

In [11]:
df.content.str.contains('support').value_counts()

True     291
False    207
Name: content, dtype: int64

In [12]:
df.content.str.contains('against').value_counts()

False    471
True      27
Name: content, dtype: int64

## Vectors

In short, they represent the "meaning" of a word as a bunch of numbers, which corresponds to a point in space. [More info.](https://www.youtube.com/watch?v=LSS_bos_TPI&list=PLRqwX-V7Uu6aQ0oh9nH8c6U1j9gCg-GdF&index=2&t=336s)

In [13]:
nlp('apple').vector

array([-3.6391e-01,  4.3771e-01, -2.0447e-01, -2.2889e-01, -1.4227e-01,
        2.7396e-01, -1.1435e-02, -1.8578e-01,  3.7361e-01,  7.5339e-01,
       -3.0591e-01,  2.3741e-02, -7.7876e-01, -1.3802e-01,  6.6992e-02,
       -6.4303e-02, -4.0024e-01,  1.5309e+00, -1.3897e-02, -1.5657e-01,
        2.5366e-01,  2.1610e-01, -3.2720e-01,  3.4974e-01, -6.4845e-02,
       -2.9501e-01, -6.3923e-01, -6.2017e-02,  2.4559e-01, -6.9334e-02,
       -3.9967e-01,  3.0925e-02,  4.9033e-01,  6.7524e-01,  1.9481e-01,
        5.1488e-01, -3.1149e-01, -7.9939e-02, -6.2096e-01, -5.3277e-03,
       -1.1264e-01,  8.3528e-02, -7.6947e-03, -1.0788e-01,  1.6628e-01,
        4.2273e-01, -1.9009e-01, -2.9035e-01,  4.5630e-02,  1.0120e-01,
       -4.0855e-01, -3.5000e-01, -3.6175e-01, -4.1396e-01,  5.9485e-01,
       -1.1524e+00,  3.2424e-02,  3.4364e-01, -1.9209e-01,  4.3255e-02,
        4.9227e-02, -5.4258e-01,  9.1275e-01,  2.9576e-01,  2.3658e-02,
       -6.8737e-01, -1.9503e-01, -1.1059e-01, -2.2567e-01,  2.41

In [14]:
apple = nlp('apple')
pear = nlp('pear')
apple.similarity(pear)

0.5968762340877272

In [15]:
truck = nlp('truck')
apple.similarity(truck)

0.18432754056976158

## Finding duplicates

In [16]:
df.shape

(498, 3)

In [17]:
df.drop_duplicates('content').shape

(473, 3)

To do more robust cleanup of duplicates, you'd want to do things to normalize the text, such as:

- [Removing extra whitespace](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.strip.html)
- [Normalizing the case](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.casefold.html)

## Classification

We want to identify which comments are in support of regulation vs. not…without reading all of them. In NLP, this is known as [text classification](https://monkeylearn.com/text-classification/).

## Classification

1. Create a training dataset.
  1. Get a subset of the data.
  1. Manually "label" it.
1. Create a model, feeding it the training dataset.
1. Apply the model to the unlabeled comments.

In [18]:
train = [
    ('I love this sandwich.', 'pos'),
    ('this is an amazing place!', 'pos'),
    ('I feel very good about these beers.', 'pos'),
    ('this is my best work.', 'pos'),
    ("what an awesome view", 'pos'),
    ('I do not like this restaurant', 'neg'),
    ('I am tired of this stuff.', 'neg'),
    ("I can't deal with this", 'neg'),
    ('he is my sworn enemy!', 'neg'),
    ('my boss is horrible.', 'neg')
]

from textblob.classifiers import NaiveBayesClassifier
cl = NaiveBayesClassifier(train)

cl.classify('this is great')

'pos'

In [19]:
cl.classify('I hate it')

'neg'

We can use a _different_ set of pre-labeled data to see how well the model works:

In [20]:
test = [
    ('the beer was good.', 'pos'),
    ('I do not enjoy my job', 'neg'),
    ("I ain't feeling dandy today.", 'neg'),
    ("I feel amazing!", 'pos'),
    ('Gary is a friend of mine.', 'pos'),
    ("I can't believe I'm doing this.", 'neg')
]

cl.accuracy(test)

0.8333333333333334

## [In-class exercise](https://colab.research.google.com/github/afeld/python-public-policy/blob/main/lecture_5_exercise.ipynb)

## Features

In [21]:
cl.show_informative_features(5)

Most Informative Features
            contains(my) = True              neg : pos    =      1.7 : 1.0
            contains(an) = False             neg : pos    =      1.6 : 1.0
             contains(I) = False             pos : neg    =      1.4 : 1.0
             contains(I) = True              neg : pos    =      1.4 : 1.0
            contains(my) = False             pos : neg    =      1.3 : 1.0


## TF-IDF

Term frequency–inverse document frequency, a.k.a.

> How specific is a given word to this particular "document" relative to the others?

[More info](https://monkeylearn.com/blog/what-is-tf-idf/)

In [22]:
nlp.Defaults.stop_words

{"'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'four',
 'from',
 'fron

In [23]:
for sentence in train:
    doc = nlp(sentence[0])
    non_stop_tokens = [token for token in doc if token.text not in nlp.Defaults.stop_words]
    print(non_stop_tokens)

[I, love, sandwich, .]
[amazing, place, !]
[I, feel, good, beers, .]
[best, work, .]
[awesome, view]
[I, like, restaurant]
[I, tired, stuff, .]
[I, deal]
[sworn, enemy, !]
[boss, horrible, .]


## [Homework 5 + 6](https://colab.research.google.com/github/afeld/python-public-policy/blob/main/hw_5.ipynb)

## Lecture 6